In [12]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
%load_ext autoreload
%autoreload 2

2024-11-14 00:29:41.447 | INFO     | train:<module>:5 - Loading dependencies - Torch...


2024-11-14 00:29:41.557 | INFO     | train:<module>:16 - Loading dependencies - Lightning...
2024-11-14 00:29:41.558 | INFO     | train:<module>:25 - Loading dependencies - Project...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from dataset.squad import CodeContestsDataModule
params = LMHyperParams(base_model_checkpoint="Qwen/Qwen2.5-0.5B", warmup_steps_count=10)
model = AutoLMFineTuner(params)
data_module = CodeContestsDataModule(16, model.tokenizer, 1024)

In [18]:
data_module.setup("fit")

2024-11-14 01:08:49.157 | INFO     | model.utils:setup:90 - Loading dataset for stage fit
2024-11-14 01:08:50.365 | INFO     | model.utils:setup:98 - Processing dataset for stage fit, workers: 1
Parameter 'function'=<bound method DollyEntityExtractionDataModule.process_samples_batch of <dataset.squad.DollyEntityExtractionDataModule object at 0x14d44a3956d0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [17]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)


Seed set to 42
2024-11-14 01:01:39.700 | INFO     | train:start_training:268 - Effective batch size: 8 (2x4)
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /admin/home-brianf/smolmodels/.venv/lib/python3.11/s ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-11-14 01:01:39.802 | INFO     | model.utils:setup:90 - Loading dataset for stage TrainerFn.FITTING
2024-11-14 01:01:41.207 | INFO     | model.utils:setup:98 - Processing dataset for stage TrainerFn.FITTING, workers: 1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4]
Loading `train_dataloader` to estimate number of stepping batches.


Configuring optimizers: 0


/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=175` in the `DataLoader` to improve performance.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

  | Name  | Type             | Params | Mode
--------------------------------------------------
0 | model | Qwen2ForCausalLM | 494 M  | eval
--------------------------------------------------
494 M     Trainable params
0         Non-trainable params
494 M     Total params
1,976.131 Total estimated model params size (MB)
0         Modules in trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=175` in the `DataLoader` to improve performance.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


  Epoch    Sample Index  Input                                               Output                                              Target
-------  --------------  --------------------------------------------------  --------------------------------------------------  --------------------------------------------------
      0               0  <|im_start|>system  Extract structured data from    <|im_start|>system  Extract structured data from    !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                         the following context in JSON form. <|im_end|>      the following context in JSON form. <|im_end|>      !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                         <|im_start|>user The ninth series of the British    <|im_start|>user The ninth series of the British    !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                         television drama series Waterloo Road began airing  television drama series Waterloo Road began airing  !!!!!!!!!!!!!!!


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined